In [1]:
import os
import glob
import pandas as pd

### SNPiR: Reliable Identification of Genomic Variants Using RNA-seq Data

In [3]:
base_dir = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/SNPiR-master/data"
hg38_reference = "%s/hg38_ucsc.fa" % base_dir
RepeatMasker = "%s/Repeats_ucsc_hg38.bed" % base_dir
# Download from UCSC table; add the first col through awk '{OFS="\t";print "1",$0}'; sort rows through sort -k3,3 -k5,5n;
gene_annotation = "%s/GENCODE_v32_modified_sorted.txt" % base_dir
rnaedit = "/home/database/RNA_editing/REDIportal/table1_full.txt"
dbsnp = ""
mills = ""
g1000 = ""
# concatenate splice junction seq(fa) with hg38 refer genome(fa);  bwa index refer_junction.fa
bwa_index = "%s/refer_junction.fa" % base_dir
picard = "java -jar /usr/share/java/picard.jar"
fq_dir = "/data4/xs/original_data/d0905-P101SC17080761-01-B1-RRL-N17/input_seq_raw_data"

#### Mapping ####

In [4]:
# align with bwa as single reads
def bwa_alignment(fq, thread_n):
    result_dir = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/bwa_result"
    result_file = os.path.join(result_dir, os.path.basename(fq).replace(".fq.gz", ".sai"))
    os.system("nohup bwa aln -t %d %s %s > %s &" % (thread_n, bwa_index, fq, result_file))

In [5]:
def generate_alignment(sai_file, sam):
    # read group information for read1 & read2, is required for GATK
    lane = os.path.basename(sai_file).split("_")[2].strip("L")
    sample_name = os.path.basename(sai_file).split(".")[0]
    RGR = "@RG\tID:%s\tLB:TrueSeq\tPL:ILLUMINA\tSM:%s" % (lane, sample_name)
    result_dir = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/bwa_result"
    sam = os.path.join(result_dir, os.path.basename(fq).replace(".sai", ".sam"))
    os.system("nohup bwa samse -n4 -r %s %s %s > %s &" % (RGR, bwa_index, sai_file, sam))

In [6]:
# merge alignment
def merge_alignment(sam1, sam2):
    result_sam = os.path.join(result_dir, os.path.basename(fq).split(".")[0] + ".sam")
    os.system("cat %s <(grep -v '^@' %s) > %s" % (sam1, sam2, result_sam))

In [7]:
# convert the position of reads that map across splicing junctions onto the genome
def convert_coordinates(merged_sam, conv_sam):
    os.system("java -Xmx4g -cp convertCoordinates < %s > %s" % (merged_sam, conv_sam))

In [ ]:
def remove_duplicates(i_sam):
    prefix = os.path.basename(i_sam).split(".")[0]
    rmdup_sam = os.path.join(result_dir, "%s_rmdup.sam" % prefix)
    metric_file = os.path.join(result_dir, "%s_out.metrics" % prefix)
    os.system("%s MarkDuplicates I=%s O=%s REMOVE_DUPLICATES=true CREATE_INDEX=true TMP_DIR=%s VALIDATION_STRINGENCY=SILENT M=%s" % (picard, i_sam, rmdup_sam, tmp_dir, metric_file))

In [21]:
def filter_unmapped_reads(i_sam):
    out_bam = os.path.join(result_dir, os.path.basename(i_sam).replace(".sam", "_mapped.bam"))
    command = "samtools view -@ 10 -bS -q 10 -F 4 %s -o %s" % (i_sam, out_bam)
    index_com = "samtools index %s" % out_bam
    os.system(command)
    os.system(index_com)

In [ ]:
def realignment():
    command = "java -Xmx16g -jar %s \
                -T RealignerTargetCreator \
                -R %s \
                -I %s \
                -o %s \
                -known %s \
                -known %s \
                -nt 8 \
                --fix_misencoded_quality_scores\
                --filter_reads_with_N_cigar"
    command_2 = "java -Xmx16g -Djava.io.tmpdir=$TMPDIR -jar `which GenomeAnalysisTK.jar` \
                -I $out_dir/merged.conv.nh.sort.rd.bam \
                -R $hg19_reference \
                -T IndelRealigner \
                -targetIntervals $out_dir/output.intervals \
                -o $out_dir/merged.conv.sort.rd.realigned.bam \
                -known $mills \
                -known $g1000 \
                --consensusDeterminationModel KNOWNS_ONLY \
                -LOD 0.4 \
                --fix_misencoded_quality_scores \
                --filter_reads_with_N_cigar"

#### Pipeline ###################

In [20]:
# def match_files():
file_dict = {}
query_word = "_1.fq.gz"
replace_word = query_word.replace("_1.", "_2.")
fq1_list = glob.glob("%s/*%s" % (fq_dir, query_word))
for fq1 in fq1_list:
    fq2 = fq1.replace(query_word, replace_word)
    prefix = os.path.basename(fq1).split("_")[0]
    file_dict[prefix] = file_dict.get(prefix, []) + [[fq1, fq2]]
# check the existence
out_files = [x for x, values in file_dict.items() for tmp_list in values for x in tmp_list if not os.path.exists(x)]
print(out_files)

[]
